In [ ]:
import gradio as gr
import torch
import re
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

c:\Users\Trung Kien\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
device_for_summary = 0 if torch.cuda.is_available() else -1


In [ ]:
MODEL_NAME = "vinai/bartpho-syllable"

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
    summarizer = pipeline(
        "summarization",
        model=model,
        tokenizer=tokenizer,
        device=device_for_summary
    )
except Exception as e:
    print(f"Lỗi tải mô hình BARTpho-syllable: {e}")
    summarizer = None
    tokenizer = None

Device set to use cpu


In [ ]:
def summarize_text(full_text):
    """
    Tóm tắt văn bản sử dụng BartPho-syllable.
    """
    if not summarizer:
        return "Mô hình tóm tắt chưa sẵn sàng."
    if not full_text.strip():
        return "Không có nội dung để tóm tắt."

    # Gọi pipeline tóm tắt
    result = summarizer(
        full_text,
        max_length=100, 
        min_length=30,
        truncation=True,
        num_beams=4,
        no_repeat_ngram_size=3,
        repetition_penalty=2.0,
        length_penalty=1.0,
        temperature=0.0,
        do_sample=False,
        early_stopping=True
    )
    key = 'summary_text' if 'summary_text' in result[0] else 'generated_text'
    return result[0][key]

In [ ]:
# 4) CHẠY THỬ MÔ HÌNH VỚI ĐOẠN VĂN BẢN MẪU
################################################################################
text = """
Hôm nay chúng ta họp để bàn về kế hoạch triển khai dự án quý tới. 
Đầu tiên, nhóm kỹ thuật sẽ cần hoàn thành việc nâng cấp hệ thống vào cuối tháng này. 
Điều này rất quan trọng để đảm bảo không ảnh hưởng đến tiến độ chung. 
Tiếp theo, nhóm kinh doanh đã liên hệ với một số đối tác và họ cần phản hồi sớm để kịp ký kết hợp đồng. 
Ngoài ra, vấn đề ngân sách cũng cần được xem xét vì có một số khoản chi chưa được phê duyệt. 
Chúng ta cần đưa ra quyết định sớm để đảm bảo nguồn lực đầy đủ. 
Cuối cùng, tôi đề xuất tổ chức một cuộc họp nhỏ vào tuần sau để kiểm tra tiến độ từng nhóm trước khi có báo cáo chính thức. 
Nếu không còn ý kiến gì thêm, chúng ta kết thúc cuộc họp tại đây.
"""

In [ ]:
summary = summarize_text(text)
print("\n📌 Văn bản gốc:")
print(text)
print("\n📌 Văn bản tóm tắt:")
print(summary)


📌 Văn bản gốc:

Hôm nay chúng ta họp để bàn về kế hoạch triển khai dự án quý tới. 
Đầu tiên, nhóm kỹ thuật sẽ cần hoàn thành việc nâng cấp hệ thống vào cuối tháng này. 
Điều này rất quan trọng để đảm bảo không ảnh hưởng đến tiến độ chung. 
Tiếp theo, nhóm kinh doanh đã liên hệ với một số đối tác và họ cần phản hồi sớm để kịp ký kết hợp đồng. 
Ngoài ra, vấn đề ngân sách cũng cần được xem xét vì có một số khoản chi chưa được phê duyệt. 
Chúng ta cần đưa ra quyết định sớm để đảm bảo nguồn lực đầy đủ. 
Cuối cùng, tôi đề xuất tổ chức một cuộc họp nhỏ vào tuần sau để kiểm tra tiến độ từng nhóm trước khi có báo cáo chính thức. 
Nếu không còn ý kiến gì thêm, chúng ta kết thúc cuộc họp tại đây.


📌 Văn bản tóm tắt:
Hôm nay chúng ta họp để bàn về kế hoạch triển khai dự án quý tới. Đầu tiên, nhóm kỹ thuật sẽ cần hoàn thành việc nâng cấp hệ thống vào cuối tháng này. Điều này rất quan trọng để đảm bảo không ảnh hưởng đến tiến độ chung. Tiếp theo, nhóm kinh doanh đã liên hệ với một số đối tác và họ

In [ ]:
# if __name__ == "__main__":
#     app = build_gradio_app()
#     app.queue()
#     app.launch(server_name="0.0.0.0", server_port=7860, share=False)

ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7860): only one usage of each socket address (protocol/network address/port) is normally permitted


OSError: Cannot find empty port in range: 7860-7860. You can specify a different port by setting the GRADIO_SERVER_PORT environment variable or passing the `server_port` parameter to `launch()`.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-large-vietnews-summarization")  
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-large-vietnews-summarization")
model.cuda()

sentence = "VietAI là tổ chức phi lợi nhuận với sứ mệnh ươm mầm tài năng về trí tuệ nhân tạo và xây dựng một cộng đồng các chuyên gia trong lĩnh vực trí tuệ nhân tạo đẳng cấp quốc tế tại Việt Nam."
text =  "vietnews: " + sentence + " </s>"
encoding = tokenizer(text, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=256,
    early_stopping=True
)
for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)


c:\Users\Trung Kien\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AssertionError: Torch not compiled with CUDA enabled